In [1]:
import pandas as pd

df = pd.read_csv('data_dac.csv', sep = ',', header = 0, error_bad_lines=False, date_parser=['posted_at'])
df = df[df.named_id.notnull() == True ]
df.nb_follower = df.nb_follower.astype('int32')
df.nb_following = df.nb_following.astype('int32')
df.length = df.length.astype('int32')
df.orthographe = df.orthographe.astype('float64')
df.reset_index(inplace = True)

b'Skipping line 75: expected 17 fields, saw 18\nSkipping line 106: expected 17 fields, saw 18\nSkipping line 138: expected 17 fields, saw 18\nSkipping line 165: expected 17 fields, saw 18\nSkipping line 166: expected 17 fields, saw 19\nSkipping line 187: expected 17 fields, saw 19\nSkipping line 261: expected 17 fields, saw 18\nSkipping line 269: expected 17 fields, saw 19\nSkipping line 339: expected 17 fields, saw 19\nSkipping line 702: expected 17 fields, saw 19\nSkipping line 731: expected 17 fields, saw 18\nSkipping line 773: expected 17 fields, saw 18\nSkipping line 844: expected 17 fields, saw 19\nSkipping line 1262: expected 17 fields, saw 18\nSkipping line 1293: expected 17 fields, saw 18\nSkipping line 1323: expected 17 fields, saw 20\nSkipping line 1577: expected 17 fields, saw 18\nSkipping line 1685: expected 17 fields, saw 19\nSkipping line 1967: expected 17 fields, saw 18\nSkipping line 1973: expected 17 fields, saw 18\nSkipping line 2102: expected 17 fields, saw 18\nSkip

Suite à l'analyse des features que nous avons effectué dans 'pretraitement' nou allons maintenant procéder à des analyses sur les textes des tweets pour voir si des tendances se dégagent et ce que l'on peut en tirer.

Nous allons commencer par définir une fonction permettant de tokenizer les tweets que nous souhaitons analyser. Nous allons d'abord supprimer tout les hyperliens et mention d'utilisateur qui risque de perturber notre analyse. Nous allons aussi supprimer les stopwords et découper les hastags en fonction des majuscules (#GiletJaune devient 'Gilet Jaune')

In [2]:
import re
import fr_core_news_md
nlp = fr_core_news_md.load()

from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from Emojilist import emojilist

In [7]:


stopwords_fr = stopwords.words('french')


def tokenize(message):
        message = re.sub(r'(?:\@|https?\://)\S+', '', message)  # remove links and @username
        message = re.sub(r'[^\w\s]', ' ', message)  # remove non-alphanumeric characters
        doc = nlp (message)
        stopWords = new_stopwords
        hashtag = False
        list = [str(token) for token in doc]
        tokens = []
        for elt in list :
            #remove stopwords
            if elt not in stopWords :
                #couper les lettres qui se repètent plus de 2fois de suite
                for i in range(len(elt)-1,1, -1):
                    if elt[i] == elt [i-1] and elt[i] == elt [i-2] :
                        elt = elt[:i]+elt[i+1:]
                #On coupe le mot si il s'agit du terme après un hastag
                if hashtag == True :
                    decoup = re.findall('[A-Z][^A-Z]*',elt)
                    for word in decoup :
                        tokens.append(word)
                    hashtag = False
                elif elt == '#' :
                    hashtag = True
                elif elt in emojilist :
                    pass
                else :
                    tokens.append(elt)
        return tokens

In [4]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import collections



def cluster_texts_Kmeans( texts, k=5):
        """ Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
        vectorizer = TfidfVectorizer(
            tokenizer=tokenize,
            max_features=1500,
            max_df=0.7,
            min_df=5,
            lowercase=True)

        tfidf_model = vectorizer.fit_transform(texts)
        km_model = KMeans(n_clusters=k, init='k-means++', n_init=10)
        km_model.fit_predict(tfidf_model)
        
        clustering = collections.defaultdict(list)

        print("Top terms per cluster:")
        order_centroids = km_model.cluster_centers_.argsort()[:, ::-1]
        #print(order_centroids)
        terms = vectorizer.get_feature_names()
        for i in range(k):
            print("\n Cluster %d:" % i)
            for ind in order_centroids[i, :10]:
                print(' %s' % terms[ind])


        

In [5]:
print(cluster_texts_Kmeans(df.text))

Top terms per cluster:

 Cluster 0:
 a
   
 les
 fait
 tout
 plus
 faire
 si
 bien
 rien

 Cluster 1:
   
 plus
 bien
 merci
 fait
 

 si
 quand
 trop
 non

 Cluster 2:
   
 les
 trop
 vrai
 via
 ah
 dit
 merci
 live
 père

 Cluster 3:
 ça
   
 si
 fait
 a
 les
 comme
 bien
 plus
 va

 Cluster 4:
 les
   
 ils
 tous
 gens
 plus
 

 là
 comme
 quand
None


Nous remarquons qu'il reste beaucoup de stopwords dans les mots les plus fréquents. Nous allons donc compléter la liste de stopwords pour qu'elle soit plus complète que la liste de base présente dans ntlk et modifier la fonction tokenize pour qu'elle prenne cette nouvelle liste

In [6]:
new_stopwords = stopwords.words('french') + ['oui','non','merci','a','bah','toujours','va','dit','alors','juste','sujet','au','la','sur','aucuns','le','ta','aussi','les','tandis',
'autre','leur','tellement','avant','là','tels','avec','ma','tes','avoir','maintenant','ton','bon','mais','tous',
'car','mes','tout','ce','mine','trop','cela','moins','très','ces','mon','tu','ceux','mot','voient','chaque','même','vont',
'ci','ni','votre','comme','nommés','vous','comment','notre','vu','dans','nous','ça','des','ou','étaient','du','où','état',
'dedans','par','étions','dehors','parce','été','depuis','pas','être','devrait','peut','doit','peu','donc','plupart',
'dos','pour','début','pourquoi','elle','quand','elles','que','en','quel','encore','quelle','essai','quelles','est','quels',
'et','qui','eu','sa','fait','sans','faites','ses','fois','seulement','font','si','hors','sien','ici','son','il','sont',
'ils','sous','je','soyez','plus']


In [8]:
print(cluster_texts_Kmeans(df.text))

Top terms per cluster:

 Cluster 0:
   
 3
 via
 ah
 jamais
 no
 malaise
 live
 père
 pareil

 Cluster 1:
   
 

 quoi
 cette
 mdrr
 faut
 2
 mdr
 jamais
 ah

 Cluster 2:
 bien
   
 sûr
 sais
 toutes
 deux
  

 ah
 connu
 aime

 Cluster 3:
 france
   
 honte
 entre
 années
 temps
 5
 
  
 3
 bientôt

 Cluster 4:
 faire
   
 vais
 faut
 plaisir
 rien
 envie
 partie
 2019
 aller
None


Maintenant on obtient des termes plus pertinents pour qualifier les clusters. Néanmoins il nous reste à définir le nombre de cluster qu'il serait pertinent de faire car pour l'instant il est fixé par nous avec la méthode des K_means.

Pour cela nous avons regardé les différents algorithme qui permettait de définir automatiquement le nombre de cluster basé sur la densité et nous avons choisi de nous interesser à la fonction DBSCAN

In [9]:
from sklearn.cluster import DBSCAN

def cluster_texts_DBSCAN( texts):
        vectorizer = TfidfVectorizer(
            tokenizer=tokenize,
            max_features=1500,
            max_df=0.7,
            min_df=5,
            lowercase=True)

        tfidf_model = vectorizer.fit_transform(texts)
        db = DBSCAN(eps=0.5, min_samples=5)
        db.fit_predict(tfidf_model)
        labels = db.labels_
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        print('Estimated number of clusters: %d' % n_clusters_)
        print('Estimated number of noise points: %d' % n_noise_)
        for j in range (n_clusters_) :
            print("\n Cluster %d:" % j)
            for i in range(len(labels)) :
                if labels[i] == j :
                    print('tweet : ' + str(df.text[i]))



In [ ]:
print(cluster_texts_DBSCAN(df.text))

L'analyse des clusters réalisé avec DBSCAN nous montre que beaucoup de point (75%) sont classés comme bruit et seul 25% sont clusterisés avec les paramètres actuels. 
Si nous voulons que plus de points soient clusterisés il faudrait que nous diminuions le paramètres eps pour DBSCAN qui catégorise a partir de quelle distance on considère 2 points comme proches. Néanmoins si l'on fait ça alors la pertinence de nos clusters risque de diminuer alors qu'elle est déjà faible pour le cluster 0 (qui n'est pourtant pas le cluster poubelle).

C'est pourquoi il faudrait mener des tests avec plusieurs valeurs de 'eps' et 'min_samples' pour voir quelle combinaison nous donne des clusters pertinents pour l'analyse du dataset.